In [1]:
import os
import re
from datetime import datetime
from dateutil import parser
import requests
from lxml.etree import HTML as html
import pandas as pd 

The location is twice in the retrieved JavaScript: 1. in the popup information and 2. in the polyline at the end. There might be more udsable data in the script.
I started from the Python package 'Get-Back-on-Your-Bike', see https://github.com/ychen306/get-back-on-your-bike 

In [2]:
# URL template for accessing spot records
SPOT_URL_TEMPL = 'http://trackleaders.com/spot/%s/%s.js'
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'
def parse(s):
    return parser.parse(s.replace('(EEST)',''))
def getSpotData(race, racer_id):
    spot_url = SPOT_URL_TEMPL% (race, racer_id)
    spot_feed = requests.get(spot_url, headers={'User-Agent': USER_AGENT}).text
    latlons=eval(spot_feed.split('LLarray = ')[1].split('; polyline')[0])
    SPOT_TS_RE_TEMPL2 = 'received at:(.*?) <br'
    timestamps = re.findall(SPOT_TS_RE_TEMPL2,
                             spot_feed,
                             re.MULTILINE)
    timestamps=[parse(e) for e in timestamps]
    return latlons,timestamps

In [3]:
race = "transconrace19"
racers = pd.read_pickle('racers.pkl')

In [4]:
dirout = str(datetime.now().date())
if not os.path.exists(dirout):
    os.mkdir(dirout)    

Write the LatLons and the timestamp to a subdirectory

In [ ]:
for racer in racers['Name']:
    try:
        latlons,timestamps = getSpotData(race,racer);
        pd.DataFrame({'LatLon': latlons,'Timestamp':timestamps}).to_pickle(dirout+'/'+racer+'.pkl')
    except:
         print(racer)   

Pawel_Pulawski
Bellamoli_Nicola
Jurgen_Mikusch
Frank_Proud
matthew_falconer
Fabian_Rabe
Ben_Davies
Stephane_OUAJA
Levente_Bagoly
Matthieu_Lifschitz
Daniel_Brayson
Peter_Boynton
Sam_Thomas
Nick_Spencer_Vellacott
Mark_Charles
Mike_Cannon
Thomas_Scherer
Lulu_Drinkwater
Felix_Newman
George_Callery
Patrick_Tschellnig
Laurent_Tesch
herve_samson
Pauline_Wassermann
Nico_Malet
T__Z___Trindade
clement_jacqueline
Brendan_Cassidy
Michael_Warren
Dennis_Wolckenhauer
Richard_Bent
Pontus_Johansson
Aidan_Allcock
David_Withers
rodolph_rainteau
THEO_DANIEL


Beginning with the 2019/07/31 the location data and the timestamp are getting out of sync.

In [ ]:
racer = 'Ben_Davies'
latlons,timestamps = getSpotData(race,'Ben_Davies');
len(latlons),len(timestamps)